# Notebook for training the YOLOv8 model

# Train the model

In [4]:
from ultralytics import YOLO

# Load a model
#model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

# Use the model
model.train(data="config.yaml", epochs=5)  # train the model
metrics = model.val()  # evaluate model performance on the validation set

#results = model("https://ultralytics.com/images/bus.jpg")  # predict on an image
#path = model.export(format="onnx")  # export the model to ONNX format

New https://pypi.org/project/ultralytics/8.0.210 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.209 🚀 Python-3.10.12 torch-1.13.1 CPU (Apple M1)
WARNING ⚠️ Upgrade to torch>=2.0.0 for deterministic training.
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=config.yaml, epochs=5, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train3, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, vis

RuntimeError: Dataset 'config.yaml' error ❌ 
Dataset 'config.yaml' images not found ⚠️, missing path '/Users/mohamadmuradi/Desktop/fix-yolo/3d_mai/notebooks/yolo_model/datasets/data/test'
Note dataset download directory is '/Users/mohamadmuradi/Desktop/fix-yolo/3d_mai/notebooks/yolo_model/datasets'. You can update this in '/Users/mohamadmuradi/Library/Application Support/Ultralytics/settings.yaml'

In [7]:
path = model.export(format="onnx")

Ultralytics YOLOv8.0.195 🚀 Python-3.11.5 torch-2.0.1 CPU (Intel Core(TM) i5-7360U 2.30GHz)

PyTorch: starting from 'runs/detect/train2/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 8, 8400) (5.9 MB)

ONNX: starting export with onnx 1.14.1 opset 17...
ONNX: export success ✅ 1.8s, saved as 'runs/detect/train2/weights/best.onnx' (11.7 MB)

Export complete (4.1s)
Results saved to /Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/notebooks/yolo_model/runs/detect/train2/weights
Predict:         yolo predict task=detect model=runs/detect/train2/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train2/weights/best.onnx imgsz=640 data=config.yaml  
Visualize:       https://netron.app


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [9]:
metrics = model.val()  # evaluate model performance on the validation set

Ultralytics YOLOv8.0.195 🚀 Python-3.11.5 torch-2.0.1 CPU (Intel Core(TM) i5-7360U 2.30GHz)
val: Scanning /Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-par
                 Class     Images  Instances      Box(P          R      mAP50  m
                   all         34         54      0.202      0.316      0.375      0.254
           210216-4x20         34         14          0          0      0.289      0.207
           210216-4x10         34         15      0.577      0.933      0.699      0.482
  10_00_0125_0084a_v4a         34         25       0.03     0.0156      0.137     0.0725
Speed: 2.7ms preprocess, 118.1ms inference, 0.0ms loss, 1.4ms postprocess per image
Results saved to runs/detect/val


# Testing

In [8]:
file_path = DATA_PATH + "data/train/images2/img_ dst4_0_1_1_moved.jpg"

#metrics = model.predict(file_path)

AttributeError: 'YOLO' object has no attribute 'test'